# New York State Supplemental Nutrition Assistance Program

### Introduction

Administered by the U.S. Department of Agriculture (USDA), the New York State Supplemental Nutrition Assistance Program (SNAP), the new name for the food stamp program, issues monthly electronic benefits that can be used like cash at authorized retail food stores. Eligibility and benefit levels are based on household size, income, assets and other factors. 

There has been a lot of attenion surrounding the SNAP since the Trump administration approved a new rule that is estimated to remove about 700,000 people from the federal SNAP program as a result of tightening of work requirements and removing individual states' discretion to create waivers for people in high employment areas. Currently, people aged between 18 and 49 who are childless and not disabled are required to work at least 20 hours a week for more than three months over a 36-hour period to qualify for food stamps. States have previously been able to create waivers for people in this group who live in high unemployment areas that may not meet these requirements. Under the rule, effective April 1, 2020, an area eligible for a waiver would have to have a 24-month average unemployment rate that is not only 20 percent above the national average but also at least 6 percent.

Using open data collected by the State of New York government (https://data.ny.gov) can take a high level look at how USDA new rules may affect SNAP recipients. According to the Center on Budget and Policy Priorities, 2,911,000 New York residents, or 15% of the state population (1 in 7) are SNAP recipients (www.cbpp.org/).

**Throughout the project we will look at three key areas:**

* SNAP and unemployment point in time analysis 
* Areas in New York we predict will see the largest impact
* The number and which areas would continue to be eligible to create waviers


### Data Cleaning 

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px


In [2]:
SNAP = pd.read_csv("supplemental-nutrition-assistance-program-snap-caseloads-and-expenditures-beginning-2002.csv")
UNEMPLOYMENT = pd.read_csv("local-area-unemployment-statistics-beginning-1976.csv")

In [3]:
SNAP.head()

,Year,Month,Month Code,District Code,District,Total SNAP Households,Total SNAP Persons,Total SNAP Benefits,Temporary Assistance SNAP Households,Temporary Assistance SNAP Persons,Temporary Assistance SNAP Benefits,Non-Temporary Assistance SNAP Households,Non-Temporary Assistance SNAP Persons,Non-Temporary Assistance SNAP Benefits
0,2019,September,9,1,Albany,17474,31806,4007529.0,5922,8142,1229277.0,11552,23664,2778252.0
1,2019,August,8,1,Albany,17509,31915,4025284.0,5982,8191,1241427.0,11527,23724,2783857.0
2,2019,July,7,1,Albany,17558,31957,4025929.0,5940,8113,1215922.0,11618,23844,2810007.0
3,2019,June,6,1,Albany,17489,31797,3985900.0,5904,8062,1204561.0,11585,23735,2781339.0
4,2019,May,5,1,Albany,17552,31820,3984954.0,5971,8119,1224905.0,11581,23701,2760049.0


In [4]:
UNEMPLOYMENT.head()

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate
0,Albany City,2019,10,47300,45200,2100,4.4
1,Albany City,2019,9,46900,44800,2100,4.4
2,Albany City,2019,8,46600,44300,2300,4.9
3,Albany City,2019,7,46800,44600,2200,4.7
4,Albany City,2019,6,47200,45200,2000,4.2


In [5]:
#We have unemployment data from 1990 - 2019 and SNAP data from 2002-2019 so we will subset so that the 
#data is even and since we don't have a full year of 2019 data we will stop at 2018

SNAP = SNAP.loc[(SNAP.Year >= 2002) & (SNAP.Year <= 2018)]
UNEMPLOYMENT = UNEMPLOYMENT.loc[(UNEMPLOYMENT.Year >= 2002) & (UNEMPLOYMENT.Year <= 2018)]

In [6]:
SNAP.info()
UNEMPLOYMENT.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11832 entries, 9 to 12353
Data columns (total 14 columns):
Year                                        11832 non-null int64
Month                                       11832 non-null object
Month Code                                  11832 non-null int64
District Code                               11832 non-null int64
District                                    11832 non-null object
Total SNAP Households                       11832 non-null int64
Total SNAP Persons                          11832 non-null int64
Total SNAP Benefits                         11832 non-null float64
Temporary Assistance SNAP Households        11832 non-null int64
Temporary Assistance SNAP Persons           11832 non-null int64
Temporary Assistance SNAP Benefits          11832 non-null float64
Non-Temporary Assistance SNAP Households    11832 non-null int64
Non-Temporary Assistance SNAP Persons       11832 non-null int64
Non-Temporary Assistance SNAP Benefits  

In [7]:
#changing data type from float to integer 
SNAP['Total SNAP Benefits'] = SNAP['Total SNAP Benefits'].astype('int64')
SNAP['Temporary Assistance SNAP Benefits'] = SNAP['Temporary Assistance SNAP Benefits'].astype('int64')
SNAP['Non-Temporary Assistance SNAP Benefits'] = SNAP['Non-Temporary Assistance SNAP Benefits'].astype('int64')

In [8]:
UNEMPLOYMENT["Area"].unique().tolist()
#This dataset gives use data at the county level, region, and metro area level. Since our SNAP data is at
#the county level we will just use county 

['Albany City',
 'Albany County',
 'Albany-Rensselaer-Schenectady Counties',
 'Albany-Schenectady-Troy Metro Area',
 'Allegany-Cattaraugus',
 'Allegany County',
 'Amherst Town',
 'Auburn City',
 'Babylon Town',
 'BALANCE OF STATE',
 'Bethlehem Town',
 'Binghamton City',
 'Binghamton Metro Area',
 'Brighton Town',
 'Bronx County',
 'Brookhaven Town',
 'Broome County',
 'Broome-Tioga',
 'Buffalo City',
 'Buffalo-Niagara Falls Metro Area',
 'Capital Region',
 'Carmel Town',
 'Cattaraugus County',
 'Cayuga-Cortland',
 'Cayuga County',
 'Central New York Region',
 'Chautauqua',
 'Chautauqua County',
 'Cheektowaga Town',
 'Chemung County',
 'Chemung-Schuyler-Steuben',
 'Chenango County',
 'Chenango-Delaware-Otsego',
 'Chili Town',
 'Cicero Town',
 'Clarence Town',
 'Clarkstown Town',
 'Clay Town',
 'Clifton Park Town',
 'Clinton County',
 'Clinton-Essex-Franklin-Hamilton',
 'Colonie Town',
 'Columbia County',
 'Columbia-Greene',
 'Cortland County',
 'Cortlandt Town',
 'Delaware County',
 'De

In [9]:
SNAP["District"].unique().tolist()

['Albany',
 'Allegany',
 'Broome',
 'Cattaraugus',
 'Cayuga',
 'Chautauqua',
 'Chemung',
 'Chenango',
 'Clinton',
 'Columbia',
 'Cortland',
 'Delaware',
 'Dutchess',
 'Erie',
 'Essex',
 'Franklin',
 'Fulton',
 'Genesee',
 'Greene',
 'Hamilton',
 'Herkimer',
 'Jefferson',
 'Lewis',
 'Livingston',
 'Madison',
 'Monroe',
 'Montgomery',
 'Nassau',
 'Niagara',
 'Oneida',
 'Onondaga',
 'Ontario',
 'Orange',
 'Orleans',
 'Oswego',
 'Otsego',
 'Putnam',
 'Rensselaer',
 'Rockland',
 'St. Lawrence',
 'Saratoga',
 'Schenectady',
 'Schoharie',
 'Schuyler',
 'Seneca',
 'Steuben',
 'Suffolk',
 'Sullivan',
 'Tioga',
 'Tompkins',
 'Ulster',
 'Warren',
 'Washington',
 'Wayne',
 'Westchester',
 'Wyoming',
 'Yates',
 'New York City']

In [10]:
#New York City is considered the county seat of these five counties: New York County, Kings County, 
#Bronx County, Richmond County, and Queens County so we need to combine those into New York City 
NYC_Counties_list = ['Kings County', 'Bronx County', 'New York County','Richmond County','Queens County']
NYC_Counties = UNEMPLOYMENT.loc[UNEMPLOYMENT['Area'].isin(NYC_Counties_list)]

In [11]:
NYC_Counties.head()
#dataset of all NYC counties 

,Area,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate
5190,Bronx County,2018,12,606200,573100,33100,5.5
5191,Bronx County,2018,11,604600,573500,31100,5.1
5192,Bronx County,2018,10,605900,573700,32200,5.3
5193,Bronx County,2018,9,601300,570300,31000,5.2
5194,Bronx County,2018,8,604500,568400,36100,6.0


In [12]:
NYC_Counties = NYC_Counties.groupby(['Year', 'Month']).sum()
NYC_Counties['Area'] = 'New York City'
NYC_Counties = NYC_Counties.reset_index() 
NYC_Counties.head()
#sum NYC counties by year and month into one area 

,Year,Month,Labor Force,Employed,Unemployed,Unemployment Rate,Area
0,2002,1,3676100,3372600,303700,41.0,New York City
1,2002,2,3712700,3408900,303800,40.6,New York City
2,2002,3,3709100,3406200,303100,40.5,New York City
3,2002,4,3695400,3403400,291800,39.3,New York City
4,2002,5,3680800,3401500,279300,38.0,New York City


In [13]:
#After combining NYC counties we can add combined data to orginal dataframe
UNEMPLOYMENT = pd.concat([NYC_Counties, UNEMPLOYMENT], ignore_index=True, sort =False)

In [14]:
UNEMPLOYMENT = UNEMPLOYMENT[~UNEMPLOYMENT.Area.isin(NYC_Counties_list)]
#Remove individual NYC counties since we joined all NYC counties previously 

In [15]:
#Subset counties and NYC only 
UNEMPLOYMENT = UNEMPLOYMENT[(UNEMPLOYMENT.Area.str.contains("County")) | (UNEMPLOYMENT['Area'] == 'New York City')]
UNEMPLOYMENT["Area"].unique().tolist()

['New York City',
 'Albany County',
 'Allegany County',
 'Broome County',
 'Cattaraugus County',
 'Cayuga County',
 'Chautauqua County',
 'Chemung County',
 'Chenango County',
 'Clinton County',
 'Columbia County',
 'Cortland County',
 'Delaware County',
 'Dutchess County',
 'Erie County',
 'Essex County',
 'Franklin County',
 'Fulton County',
 'Genesee County',
 'Greene County',
 'Hamilton County',
 'Herkimer County',
 'Jefferson County',
 'Lewis County',
 'Livingston County',
 'Madison County',
 'Monroe County',
 'Montgomery County',
 'Nassau County',
 'Niagara County',
 'Oneida County',
 'Onondaga County',
 'Ontario County',
 'Orange County',
 'Orleans County',
 'Oswego County',
 'Otsego County',
 'Putnam County',
 'Rensselaer County',
 'Rockland County',
 'Saratoga County',
 'Schenectady County',
 'Schoharie County',
 'Schuyler County',
 'Seneca County',
 'Steuben County',
 'St. Lawrence County',
 'Suffolk County',
 'Sullivan County',
 'Tioga County',
 'Tompkins County',
 'Ulster C

In [16]:
UNEMPLOYMENT["Area"].nunique() == SNAP["District"].nunique()
#confirm our county count is accurate

True

### Counties Trends of SNAP Recipients and Unemployed Residents Over Time

In [30]:
df_snap = SNAP.groupby(["District","Year"]).sum()
df_snap = df_snap.reset_index() 

fig = px.line(df_snap, x="Year", y="Total SNAP Persons", color="District",
              line_group="District", hover_name="District")
fig.update_layout(title_text='New York Counties SNAP Trend from 2002-2019', showlegend=False,
                   yaxis_title= "Total SNAP Participants in Millions", xaxis_title="Year", xaxis_rangeslider_visible=True)
fig.show()

In [18]:
UNEMPLOYMENT3 = pd.read_csv("local-area-unemployment-statistics-beginning-1976.csv")
UNEMPLOYMENT3 = UNEMPLOYMENT3[(UNEMPLOYMENT3.Area.str.contains("County"))]
UNEMPLOYMENT_BY_Year = UNEMPLOYMENT3.groupby(["Area", "Year"]).sum()
UNEMPLOYMENT_BY_Year = UNEMPLOYMENT_BY_Year.reset_index() 

fig = px.line(UNEMPLOYMENT_BY_Year, x="Year", y="Unemployed", color="Area",
              line_group="Area", hover_name="Area")
fig.update_layout(title_text='New York Counties Unemployment Trend from 1990-2019', showlegend=False,
                   yaxis_title= "Unemployed Members in Millions", xaxis_title="Year", xaxis_rangeslider_visible=True)
fig.show()

### Temporary Assistance  vs Non-Temporary Assistance  


For many people temporary assistance SNAP benefits provides assistance during a fairly brief stretch of hard times, such as unemployment. For millions of others – foremost, the elderly, disabled, and those with dependents, – the program is a long-term support. More than 55% of New York SNAP participants are in families with children and almost 46% are in families with members who are elderly or disabled. For most SNAP participants, there are no limits on the number of times or on the total amount of time they can receive benefits as long as they meet the they can participate in the program or on the total amount of time they can receive benefits as long as they meet the eligibility requirements. As a result, recipients whose financial circumstances and other needs fluctuate over time may have multiple spells of participation (Dynamics of Supplemental Nutrition Assistance Program Participation from 2008 to 2012, USDA).


In [19]:
#create dataframe that sums our total SNAP data by year and month
SNAP_BY_YEAR = SNAP.groupby(["Year"]).sum()
SNAP_BY_YEAR = SNAP_BY_YEAR.reset_index() 
SNAP_BY_YEAR

,Year,Month Code,District Code,Total SNAP Households,Total SNAP Persons,Total SNAP Benefits,Temporary Assistance SNAP Households,Temporary Assistance SNAP Persons,Temporary Assistance SNAP Benefits,Non-Temporary Assistance SNAP Households,Non-Temporary Assistance SNAP Persons,Non-Temporary Assistance SNAP Benefits
0,2002,4524,20628,8239418,16265748,1520823589,5702540,10095319,1032987771,2536878,6170429,487835818
1,2003,4524,20628,8827721,17634758,1742027971,6015172,10743616,1164505897,2812549,6891142,577522074
2,2004,4524,20628,10034110,19752895,1965320388,6778517,11910060,1279220913,3255593,7842835,686099475
3,2005,4524,20628,11062231,21213360,2188400866,6939810,11587438,1242813319,4122821,9625922,945587547
4,2006,4524,20628,11271022,21465995,2281071170,7001595,11570197,1280724098,4269427,9895798,1000347072
5,2007,4524,20628,11515370,21764190,2379991528,6500412,10837302,1233683858,5014958,10926888,1146307670
6,2008,4524,20628,12919782,24355072,2801699099,6808360,10397381,1251561912,6111422,13957691,1550137187
7,2009,4524,20628,15560415,29343602,4341851063,7131771,10811283,1730698905,8428644,18532319,2611152158
8,2010,4524,20628,18075370,34020897,5155293674,7494878,11326226,1883328473,10580492,22694671,3271965201
9,2011,4524,20628,19415673,36352331,5451671131,7706059,11599642,1918357032,11709614,24752689,3533314099


In [20]:
df = SNAP_BY_YEAR



fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Year, y=df['Temporary Assistance SNAP Benefits'], name="Temporary Assistance SNAP Benefits",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=df.Year, y=df['Non-Temporary Assistance SNAP Benefits'], name="Non-Temporary Assistance SNAP Benefits",
                         line_color='dimgray'))

fig.update_layout(title_text='Temporary Assistance SNAP Benefits vs Non-Temporary Assistance SNAP Benefits',
                  xaxis_rangeslider_visible=True, yaxis_title="Total Amount in Billions", xaxis_title="Year",
)
fig.show()

In [21]:
df = SNAP_BY_YEAR

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Year, y=df['Temporary Assistance SNAP Households'], name="Temporary Assistance SNAP Households",
                         line_color='deepskyblue'))

fig.add_trace(go.Scatter(x=df.Year, y=df['Non-Temporary Assistance SNAP Households'], name="Non-Temporary Assistance SNAP Households",
                         line_color='dimgray'))

fig.update_layout(title_text='Temporary Assistance SNAP Households vs Non-Temporary Assistance SNAP Households',
                  xaxis_rangeslider_visible=True, yaxis_title="Total Number in Millions", xaxis_title="Year",
)
fig.show()

### 2019  Counties Trends of SNAP Recipients and Unemployed Residents


In [22]:
#create dataframe that sums our total 2018 SNAP data by district
SNAP_BY_Area = SNAP.loc[(SNAP.Year == 2019)]
SNAP_BY_Area = SNAP.groupby(["District"]).sum()
SNAP_BY_Area = SNAP_BY_Area.reset_index() 
SNAP_BY_Area = SNAP_BY_Area.loc[:,['District', 'Total SNAP Households', 'Total SNAP Persons', 'Total SNAP Benefits']]
SNAP_BY_Area = SNAP_BY_Area.nlargest(10, ['Total SNAP Persons']) 

#sort_values("Total SNAP Persons", axis = 0, ascending = False, inplace = True) 
SNAP_BY_Area

,District,Total SNAP Households,Total SNAP Persons,Total SNAP Benefits
28,New York City,161330969,293476455,41038508747
13,Erie,13123170,25239602,3068267899
25,Monroe,10275679,19852521,2427652139
47,Suffolk,9142000,15876632,2099145957
55,Westchester,6752584,12382034,1606611205
31,Onondaga,5751597,11599643,1395632872
27,Nassau,5529184,9035019,1180309318
33,Orange,2911422,6946590,862859091
30,Oneida,3259819,6699278,787221422
39,Rockland,2345469,6672471,823365236


In [23]:

df = SNAP_BY_Area

fig = go.Figure(go.Bar(
            x= df['Total SNAP Persons'],
            y= df.District,
            orientation='h'))
fig.update_layout(title_text='Top 10 Counties Total SNAP Persons Counties in 2019',
                   yaxis_title="Counties", xaxis_title="Total Number in Millions",
)

fig.show()


In [24]:
#create dataframe that sums our total 2019 UNEMPLOYMENT data by district


UNEMPLOYMENT2 = pd.read_csv("local-area-unemployment-statistics-beginning-1976.csv")
UNEMPLOYMENT2 = UNEMPLOYMENT2[(UNEMPLOYMENT2.Area.str.contains("County"))]
UNEMPLOYMENT2
UNEMPLOYMENT_BY_Area = UNEMPLOYMENT2.loc[(UNEMPLOYMENT2.Year == 2019)]
UNEMPLOYMENT_BY_Area = UNEMPLOYMENT2.groupby(["Area"]).sum()
UNEMPLOYMENT_BY_Area = UNEMPLOYMENT_BY_Area.reset_index() 
UNEMPLOYMENT_BY_Area
UNEMPLOYMENT_BY_Area = UNEMPLOYMENT_BY_Area.loc[:,['Area', 'Labor Force', 'Employed', 'Unemployed']]
UNEMPLOYMENT_BY_Area = UNEMPLOYMENT_BY_Area.nlargest(10, ['Labor Force']) 
UNEMPLOYMENT_BY_Area

,Area,Labor Force,Employed,Unemployed
23,Kings County,385227300,354746100,30482100
40,Queens County,381732400,356655800,25077400
30,New York County,309931200,290174700,19757400
51,Suffolk County,268618200,254646200,13972500
29,Nassau County,247044300,235538900,11504600
2,Bronx County,186851200,169508800,17342200
59,Westchester County,168087700,160091500,7996100
14,Erie County,166348900,156706700,9643700
27,Monroe County,134542600,127815100,6728400
33,Onondaga County,83309000,79057200,4252000


In [25]:
df = UNEMPLOYMENT_BY_Area

x = df['Area']
fig = go.Figure(go.Bar(x=x, y=df['Employed'], name='Employed'))
fig.add_trace(go.Bar(x=x, y=df['Unemployed'], name='Unemployed'))

fig.update_layout(title_text='Top 10 Counties Unemploymed vs Employed in 2019',
                   yaxis_title="Total Number in Millions", xaxis_title="Top 10 Counties",barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

### Areas Eligible to Create Waviers with New SNAP Rules

In [26]:
#creating dataframe for counties who experienced an unemployment rate of 6.0 or higher in the last year 


UNEMPLOYMENT3 = UNEMPLOYMENT3 = pd.read_csv("local-area-unemployment-statistics-beginning-1976.csv")
UNEMPLOYMENT3 = UNEMPLOYMENT3.loc[(UNEMPLOYMENT3.Area.str.contains("County"))]
UNEMPLOYMENT3 = UNEMPLOYMENT3.loc[(UNEMPLOYMENT3.Year == 2019)]
UNEMPLOYMENT3.columns = UNEMPLOYMENT3.columns.str.replace(' ', '')
UNEMPLOYMENT3 = UNEMPLOYMENT3.loc[(UNEMPLOYMENT3.UnemploymentRate >= 6.0)]

In [27]:
fig = px.bar(UNEMPLOYMENT3, x='Area', y='UnemploymentRate')
fig.update_layout(title_text='Potential Counties Impacted by SNAP New Rules',
                   yaxis_title="", xaxis_title="Counties")

fig.show()

In [28]:
UNEMPLOYMENT_Impact = UNEMPLOYMENT3.groupby(["Area"]).UnemploymentRate.mean().to_frame()
UNEMPLOYMENT_Impact = UNEMPLOYMENT_Impact.reset_index()
UNEMPLOYMENT_Impact
#average of areas with a unemployment rate of => 6.0

,Area,UnemploymentRate
0,Allegany County,6.333333
1,Bronx County,6.300000
2,Cattaraugus County,6.000000
3,Cortland County,6.200000
4,Hamilton County,7.975000
5,Jefferson County,6.533333
6,Lewis County,7.100000
7,Oswego County,6.350000
8,Schoharie County,6.200000
9,Schuyler County,6.150000


In [29]:
fig = px.scatter(UNEMPLOYMENT_Impact, x="UnemploymentRate", y="Area",
                 title="Potential Counties Impacted by SNAP New Rules"                )

fig.show()

### Conclusion

During my research, I've learned that majority of the population on food stamps benefit will not be impacted by the new rules put in place by USDA because this population consist of children, seniors, adults caring for a dependent, or those unable to work.  As a result, childless adults are a small, but highly vulnerable, segment of SNAP participants, accounting for about 10% of all SNAP participants. I will be continuing research on this topic and looking more into detail on the characteristics of those in the small population who may be impacted by new rules. I predict that these individuals are low-income citizens in low paying fields that have little to no job stability. I also would like to look at unemployment data that specifies the number of hours worked and the duration throughout the year.

Overall, I believe this policy will have a negative impact on those struggling to find and maintain a job. Although our economy is experiencing growth, there is still a huge population working in low-income positions and struggling to qualify for the positions in fields with vast openings. Those in these low-income positions often look for public assistance programs such as SNAP to support themselves. We cannot eliminate programs such as SNAP without offering opportunities that will help low-income workers quality for higher paying jobs and therefore putting them in positions to no longer depend on programs like SNAP.


In [31]:
jupyter nbconvert my_notebook.ipynb --to html --output output.html


SyntaxError: invalid syntax (<ipython-input-31-24c32c254453>, line 1)